In [1]:
using Gurobi, JuMP

# Naive formulation (Kantorovich Formulation)

In [5]:
W = 100
w = [25,35,45]
n = [10,20,30]
m = 3
l = 25

model_naive = Model(with_optimizer(Gurobi.Optimizer))
#set_optimizer_attribute(model_naive, "Presolve", 0)
#set_optimizer_attribute(model_naive, "Heuristics", 0)
#set_optimizer_attribute(model_naive, "Cuts", 0)

@variable(model_naive,y[1:l],Bin)
@variable(model_naive,x[1:m,1:l]>=0)

@objective(model_naive,Min,sum(y))

@constraint(model_naive,sum(x,dims=2).>=n)
@constraint(model_naive,sum(transpose(x.*w),dims=2).<=W*y)

optimize!(model_naive)
value.(x)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 28 rows, 100 columns and 175 nonzeros
Model fingerprint: 0xb9de3188
Variable types: 75 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Presolve time: 0.00s
Presolved: 28 rows, 100 columns, 175 nonzeros
Variable types: 75 continuous, 25 integer (25 binary)

Root relaxation: objective 2.300000e+01, 38 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      23.0000000   23.00000  0.00%     -    0s

Explored 0 nodes (38 simplex iterations) in 0.00 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 23 



3×25 Array{Float64,2}:
 0.0      0.0      0.0      0.0      …  0.0      0.0  0.0      0.0
 0.0      0.0      0.0      0.0         2.85714  0.0  2.85714  0.0
 2.22222  2.22222  2.22222  2.22222     0.0      0.0  0.0      2.22222

# Pattern based formulation (small problem)

In [77]:
W = 100
w = [25,35,45]
n = [10,20,30]
m = 3

model_pat_s = Model(with_optimizer(Gurobi.Optimizer))
set_optimizer_attribute(model_pat_s, "Presolve", 0)
set_optimizer_attribute(model_pat_s, "Heuristics", 0)
set_optimizer_attribute(model_pat_s, "Cuts", 0)

a = [1 0 0 2 1 1 0 3 0 2 0 2 1 4;
     0 1 0 0 1 0 2 0 1 1 0 0 2 0;
     0 0 1 0 0 1 0 0 1 0 2 1 0 0]
    
l = size(a[1,:])[1]

@variable(model_pat_s,z[1:l]>=0,Int)
@objective(model_pat_s,Min,sum(z))
@constraint(model_pat_s,(a*z).>=n)
optimize!(model_pat_s)
value.(z)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3 rows, 14 columns and 20 nonzeros
Model fingerprint: 0xb4db470e
Variable types: 0 continuous, 14 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Variable types: 0 continuous, 14 integer (0 binary)

Root relaxation: objective 2.500000e+01, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      25.0000000   25.00000  0.00%     -    0s

Explored 0 nodes (3 simplex iterations) in 0.00 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 25 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.500000000000e+0

14-element Array{Float64,1}:
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
  0.0
 -0.0
 -0.0
 -0.0
 15.0
 -0.0
 10.0
 -0.0

# Pattern based formulation (large problem)

In [141]:
########
# data #
########

W = 5600
w = [1380,1520,1560,1710,1820,1880,1930,2000,2050,2100,2140,2150,2200]
n = [22,25,12,14,18,18,20,10,12,14,16,18,20]
m = 13

#W = 9
#w = [2,3,4,5,6,7,8]
#n = [4,2,6,6,2,2,2]
#m = 7

##########################
# build initial patterns #
##########################

using LinearAlgebra 
a = Matrix{Float64}(I, m, m)

#a = [4 0 0 0 0 0 0; 0 3 0 0 0 0 0; 0 0 2 0 0 0 0; 0 0 0 1 0 0 0; 0 0 0 0 1 0 0; 0 0 0 0 0 1 0; 0 0 0 0 0 0 1]

#################################
# find new interesting patterns #
#################################

l=m
while true

    # define pattern problem with initial patterns
    model_pat_l = Model(with_optimizer(Gurobi.Optimizer))
    set_optimizer_attribute(model_pat_l, "Presolve", 0)
    set_optimizer_attribute(model_pat_l, "Heuristics", 0)
    set_optimizer_attribute(model_pat_l, "Cuts", 0)
    set_silent(model_pat_l)
    @variable(model_pat_l,z[1:l]>=0)
    @objective(model_pat_l,Min,sum(z))
    @constraint(model_pat_l,con_pat,(a*z).>=n)
    optimize!(model_pat_l)

    #define knapsack problem
    model_kp = Model(with_optimizer(Gurobi.Optimizer))
    set_optimizer_attribute(model_kp, "Presolve", 0)
    set_optimizer_attribute(model_kp, "Heuristics", 0)
    set_optimizer_attribute(model_kp, "Cuts", 0)
    set_silent(model_kp)
    @variable(model_kp,p[1:m]>=0,Int)
    lambda = zeros(m)
    for i = 1:m
        lambda[i] = dual(con_pat[i])
    end
    @objective(model_kp,Max,sum(lambda.*p))
    @constraint(model_kp,sum(p.*w)<=W)
    optimize!(model_kp)
    
    println(value.(p))
    
    if objective_value(model_kp)-1 <= 0
        break
    end
    
    l=l+1
    a = [a value.(p)]
end

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
[4.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
[0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
[-0.0, 3.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
[-0.0, -0.0, -0.0, -0.0, 3

In [158]:
println()
print("added : ")
println(l-m)

println()
print("solution (relaxed) : \n")
zs = value.(z)
for z_val in zs
    println(z_val)
end
println()

print("objective value (relaxed) : ")
print(objective_value(model_pat_l))

println()
print("solution (rounding up) : \n")
zs_round = round.(zs,RoundUp)
for z_val in zs_round
    println(z_val)
end
println()

print("objective value (rounding up) : ")
println(sum(zs_round))


added : 30

solution (relaxed) : 
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.25000000000000044
0.0
0.0
0.0
0.0
0.0
0.0
7.0
0.0
0.0
0.0
0.0
0.0
1.3333333333333353
0.0
0.0
2.999999999999999
7.000000000000002
0.0
7.999999999999998
8.000000000000002
3.666666666666666
3.9999999999999982
10.0
0.0
0.0
4.999999999999998
3.6666666666666665
0.0
12.000000000000002

objective value (relaxed) : 72.91666666666667
solution (rounding up) : 
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
7.0
0.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0
3.0
8.0
0.0
8.0
9.0
4.0
4.0
10.0
0.0
0.0
5.0
4.0
0.0
13.0

objective value (rounding up) : 78.0
